In [1]:

from skimage.measure import label

#Scientific computing 
import numpy as np
import os
#Pytorch packages
import torch
from torch import nn
import torch.optim as optim
import torchvision
#from torchvision import datasets
from tensorboardX import SummaryWriter
#Visulization
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image
#Others
#from torch.utils.data import DataLoader, Subset
from torch.autograd import Variable
import matplotlib.pyplot as plt
import copy
#from finetuneSAM.utils.dataset import Public_dataset
import torch.nn.functional as F
from torch.nn.functional import one_hot
from pathlib import Path
from tqdm import tqdm
#from finetuneSAM.utils.losses import DiceLoss
#from finetuneSAM.utils.dsc import dice_coeff_multi_class
import cv2
import monai
#from finetuneSAM.utils.utils import vis_image
#import cfg
import json



# General
import numpy as np
import torch


from skimage.measure import label

#Scientific computing 
import numpy as np
import os
#Pytorch packages
import torch

import torchvision


import matplotlib.pyplot as plt

from PIL import Image


import matplotlib.pyplot as plt
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [3]:
#Mobile Sam
from MobileSAM.mobile_sam import SamAutomaticMaskGenerator, SamPredictor, sam_model_registry

sam_checkpoint = "mobile_sam.pt"
model_type = "vit_t"

mobile_sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
mobile_sam = mobile_sam.to(device=device)
mobile_sam.eval()

predictor = SamPredictor(mobile_sam)

ModuleNotFoundError: No module named 'mobile_sam'

In [ ]:
#Fine-Tune Sam
from finetuneSAM.models.sam import SamAutomaticMaskGenerator, SamPredictor, sam_model_registry
from finetuneSAM.cfg import parse_args
args =  parse_args()


# setting if_mask_decoder_adapter = True puts adapters inside 2-way transformer blocks
# this does not change the number of decoder 2-way transformer blocks (def = 2)
# decoder_adapt_depth denotes how many of the two 2-way transformer blocks are adapted


# setting if_encoder_adapter = True puts adapters inside TinyViTBlocks in the encoder
# this does not change the number of encoder TinyViTBlocks (def = 4)
# encoder_adapt_depth (e.g. [1,2]) denotes how deep blocks will be adapted

#args.finetune_type = "vanilla"
args.finetune_type = "adapter"# "vanilla"
args.if_mask_decoder_adapter = True
#args.image_size = 512
#args.decoder_adapt_depth = 1
args.num_cls = 3


# Load the pre-trained model
sam_checkpoint = "mobile_sam.pt"
# Define the model type: Tiny Vit
model_type = "vit_t"

# Load the model
mobile_sam_f = sam_model_registry[model_type](args , checkpoint=sam_checkpoint)
# Move the model to the device
mobile_sam_f = mobile_sam_f.to(device=device)
# Set the model to evaluation mode
#mobile_sam_f.eval()

predictor_f = SamPredictor(mobile_sam_f)

c:\Users\newma\AppData\Local\Programs\Python\Python310\lib\site-packages\timm\models\layers\__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
c:\Users\newma\AppData\Local\Programs\Python\Python310\lib\site-packages\timm\models\registry.py:4: FutureWarning: Importing from timm.models.registry is deprecated, please import via timm.models
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.models", FutureWarning)
c:\KU\ATIA\finetuneSAM\models\sam\modeling\tiny_vit_sam.py:760: UserWarning: Overwriting tiny_vit_5m_224 in registry with finetuneSAM.models.sam.modeling.tiny_vit_sam.tiny_vit_5m_224. This is because the name being registered conflicts with an existing name. Please check if this is not expected.
  return register_model(fn_wrapper)
c:\KU\ATIA\finetuneSAM\models\sam\modeling\tiny_vit_sam.p

In [7]:
#mobile_sam_f.mask_decoder = mobile_sam.mask_decoder

In [13]:
image = Image.open('picture4.jpg')
image = np.asarray(image)
image = image[0:254,0:254]

p = None
predictor.set_image(image)
masks, scores, logits = predictor.predict(p,point_labels= np.array([1]))
scores

array([0.4669404 , 0.7694236 , 0.96506643], dtype=float32)

In [6]:
image = Image.open('picture4.jpg')
print(image.size)
image = np.asarray(image)
#image = image[0:254,0:254]
image = image[0:1024,0:1024]
image.shape

p = None
predictor_f.set_image(image)
masks, scores, logits = predictor_f.predict(p,point_labels= np.array([1]), multimask_output=True)
scores


img_emb = mobile_sam_f.image_encoder(torch.tensor(image).permute(2,0,1).cuda())

(2048, 1365)


c:\KU\ATIA\finetuneSAM\models\sam\predictor.py:89: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_image = torch.tensor(transformed_image*1.0/255)


RuntimeError: Input type (torch.cuda.ByteTensor) and weight type (torch.cuda.FloatTensor) should be the same

In [53]:
mp = mobile_sam.parameters()
mpf = mobile_sam_f.parameters()

numb_of_extra_params = 0

mob = mobile_sam.state_dict()
fine = mobile_sam_f.state_dict()
for name, param in mob.items():
    if name in fine:
        if not torch.equal(param, fine[name]):
            print("is in both but not equal: ", name)
    else:
        print("is in mobile_sam but not in mobile_sam_f: ", name)

for name, param in fine.items():
    if name not in mob:
        
        print("extra tensor(grad ",param.requires_grad,"): ", name)
        numb_of_extra_params += param.numel()

print("Number of extra parameters in fine-tuned model: ", numb_of_extra_params)


extra tensor(grad  False ):  mask_decoder.transformer.layers.0.MLP_Adapter.D_fc1.weight
extra tensor(grad  False ):  mask_decoder.transformer.layers.0.MLP_Adapter.D_fc1.bias
extra tensor(grad  False ):  mask_decoder.transformer.layers.0.MLP_Adapter.D_fc2.weight
extra tensor(grad  False ):  mask_decoder.transformer.layers.0.MLP_Adapter.D_fc2.bias
extra tensor(grad  False ):  mask_decoder.transformer.layers.0.Adapter.D_fc1.weight
extra tensor(grad  False ):  mask_decoder.transformer.layers.0.Adapter.D_fc1.bias
extra tensor(grad  False ):  mask_decoder.transformer.layers.0.Adapter.D_fc2.weight
extra tensor(grad  False ):  mask_decoder.transformer.layers.0.Adapter.D_fc2.bias
extra tensor(grad  False ):  mask_decoder.transformer.layers.1.MLP_Adapter.D_fc1.weight
extra tensor(grad  False ):  mask_decoder.transformer.layers.1.MLP_Adapter.D_fc1.bias
extra tensor(grad  False ):  mask_decoder.transformer.layers.1.MLP_Adapter.D_fc2.weight
extra tensor(grad  False ):  mask_decoder.transformer.laye

In [49]:
fine = mobile_sam_f.state_dict()

for name, param in fine.items():        
        if param.requires_grad:
            print("Req grad: ", name)
        


In [60]:
numb_of_params = 0

x = mobile_sam_f.mask_decoder.state_dict()
for name, param in x.items():
    #print(name)
    numb_of_params += param.numel()
numb_of_params/1e6

4.124516

In [4]:


def load_data(path, num_points):
    batches = num_points//batch_size
    out = torch.zeros((batches,batch_size,3,1024,1024))
    in_batch = 0
    batch_num = 0
    for i in range (num_points):
        image = torchvision.io.read_image(path +"/"+ str(i) + ".png")
        image = F.pad(input=image, pad=(256, 256, 256, 256), mode='constant', value=0)
        out[batch_num][in_batch] = image
        in_batch += 1
        if in_batch == batch_size:
            in_batch = 0
            batch_num += 1
    return out
        
test_imgs = load_data("data/Data/test/image", val_points)
test_masks = load_data("data/Data/test/mask", val_points)
train_imgs = load_data("data/Data/train/image", train_points)
train_masks = load_data("data/Data/train/mask", train_points)

In [31]:

import torch.nn.functional as F
train_points = 80
val_points = 20
checkpoints_path  = "checkpoints"
batch_size = 5

# TRAIN PATHS
# data\Data\train\image
# data\Data\train\mask

# TEST PATHS
# data\Data\test\image
# data\Data\test\mask

def load_data(path, num_points, binary = False):
    if binary:
        out = torch.zeros((num_points,1024,1024,1))
    else:
        out = torch.zeros((num_points,1024,1024,3))
    for i in range (num_points):
        p = path +"/"+ str(i) + ".png"
        image = Image.open(p)
        print(image.size)
        image = np.asarray(image)
        if binary:
            image = image.reshape(image.shape[0],image.shape[1],1)
        
        image = np.pad(image, ((256, 256), (256, 256), (0, 0)), 'constant')
        image = torch.from_numpy(image)
        #print(image.size())
        #image = torchvision.io.read_image(path +"/"+ str(i) + ".png")
        #image[0] = F.pad(input=image[0], pad=(256, 256, 256, 256), mode='constant', value=0)
        #image[1] = F.pad(input=image[1], pad=(256, 256, 256, 256), mode='constant', value=0)
        #image = image.permute(1,2,0)

        out[i] = image
    return out
test_masks = load_data("data/Data/test/mask", val_points, binary = True)    
test_imgs = load_data("data/Data/test/image", val_points)

train_imgs = load_data("data/Data/train/image", train_points)
train_masks = load_data("data/Data/train/mask", train_points, binary = True)

(512, 512)


TypeError: can't assign a numpy.ndarray to a torch.FloatTensor

In [28]:
print(train_imgs[0].shape)
predictor_f.set_image(train_imgs[0])

torch.Size([1024, 1024, 3])


ValueError: pic should not have > 4 channels. Got 1024 channels.

In [13]:


image = torchvision.io.read_image("data/Data/train/image/0.png")


import torch.nn.functional as F

image_pad = F.pad(input=image, pad=(256, 256, 256, 256), mode='constant', value=0)
image_pad.shape

torch.Size([3, 1024, 1024])

In [36]:
def dice_coeff_multi_class(pred, target, n_classes):
    """Calculate the mean Dice Coefficient for multi-class data."""
    dice_scores = []
    for cls in range(n_classes):  # Iterate over each class
        pred_cls = (pred == cls).long()  # Binary mask for current predicted class
        target_cls = (target == cls).long()  # Binary mask for current actual class

        smooth = 1.0
        intersection = (pred_cls & target_cls).float().sum((1, 2))  # Sum over height and width dimensions
        union = pred_cls.float().sum((1, 2)) + target_cls.float().sum((1, 2))
        
        dice = (2. * intersection + smooth) / (union + smooth)
        dice_scores.append(dice)
    
    return torch.stack(dice_scores).mean() 

In [29]:


sam = mobile_sam_f

data = {'image':train_imgs, 'mask':train_masks}


if args.finetune_type == 'adapter':
    for n, value in sam.named_parameters():
        if "Adapter" not in n: # only update parameters in adapter
            value.requires_grad = False
    print('if update encoder:',args.if_update_encoder)
    print('if image encoder adapter:',args.if_encoder_adapter)
    print('if mask decoder adapter:',args.if_mask_decoder_adapter)
    if args.if_encoder_adapter:
        print('added adapter layers:',args.encoder_adapter_depths)
    
# elif args.finetune_type == 'vanilla' and args.if_update_encoder==False:   
#     print('if update encoder:',args.if_update_encoder)
#     for n, value in sam.image_encoder.named_parameters():
#         value.requires_grad = False
# elif args.finetune_type == 'lora':
#     print('if update encoder:',args.if_update_encoder)
#     print('if image encoder lora:',args.if_encoder_lora_layer)
#     print('if mask decoder lora:',args.if_decoder_lora_layer)
#     sam = LoRA_Sam(args,sam,r=4).sam
# sam.to('cuda')
b_lr = args.lr
epochs = 80
    
optimizer = optim.AdamW(sam.parameters(), lr=b_lr, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.1, amsgrad=False)
optimizer.zero_grad()
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5) #learning rate decay
criterion1 = monai.losses.DiceLoss(sigmoid=True, squared_pred=True, to_onehot_y=True,reduction='mean')
criterion2 = nn.CrossEntropyLoss()
iter_num = 0
max_iterations = epochs * train_points
writer = SummaryWriter(checkpoints_path + '/log')

pbar = tqdm(range(epochs))
val_largest_dsc = 0
last_update_epoch = 0
for epoch in pbar:


    sam.train()
    train_loss = 0
    # batches here?
    #for i,_ in enumerate(tqdm(data['image'])):
    for i in range(1):
        imgs = data['image'][0:1].cuda()
        msks = torchvision.transforms.Resize((args.out_size,args.out_size))(data['mask'][0:1])
        msks = msks.cuda()



        if args.if_update_encoder:
            img_emb = sam.image_encoder(imgs)
        else:
            with torch.no_grad():
                img_emb = sam.image_encoder(imgs)
        
        # get default embeddings
        sparse_emb, dense_emb = sam.prompt_encoder(
            points=None,
            boxes=None,
            masks=None,
        )
        pred, _ = sam.mask_decoder(
                        image_embeddings=img_emb,
                        image_pe=sam.prompt_encoder.get_dense_pe(), 
                        sparse_prompt_embeddings=sparse_emb,
                        dense_prompt_embeddings=dense_emb, 
                        #multimask_output=True,
                        multimask_output=False,
                        )
        
        print("THIS WORKS")
        print(msks.shape)
        print(msks.float().shape)
        print(pred.shape)
        


        #loss_dice = criterion1(pred,msks.float()) 
        loss_ce = criterion2(pred,torch.squeeze(msks.long(),1))
        loss =  loss_ce
        #loss =  loss_dice + loss_ce
        
        loss.backward()
        optimizer.step()
        optimizer.zero_grad(set_to_none=True)
        
        if args.if_warmup and iter_num < args.warmup_period:
            lr_ = args.lr * ((iter_num + 1) / args.warmup_period)
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr_

        else:
            if args.if_warmup:
                shift_iter = iter_num - args.warmup_period
                assert shift_iter >= 0, f'Shift iter is {shift_iter}, smaller than zero'
                lr_ = args.lr * (1.0 - shift_iter / max_iterations) ** 0.9  # learning rate adjustment depends on the max iterations
                for param_group in optimizer.param_groups:
                    param_group['lr'] = lr_
            else:
                lr_ = args.lr

        train_loss += loss.item()
        iter_num+=1
        writer.add_scalar('info/lr', lr_, iter_num)
        writer.add_scalar('info/total_loss', loss, iter_num)
        writer.add_scalar('info/loss_ce', loss_ce, iter_num)
        writer.add_scalar('info/loss_dice', loss_dice, iter_num)

    train_loss /= (i+1)
    pbar.set_description('Epoch num {}| train loss {} \n'.format(epoch,train_loss))

    # if epoch%2==0:
    #     eval_loss=0
    #     dsc = 0
    #     sam.eval()
    #     with torch.no_grad():
    #         for i in tqdm(data['image']):
    #             imgs = data['image'][i].cuda()
    #             msks = torchvision.transforms.Resize((args.out_size,args.out_size))(data['mask'][i])
    #             msks = msks.cuda()

    #             img_emb= sam.image_encoder(imgs)
    #             sparse_emb, dense_emb = sam.prompt_encoder(
    #                 points=None,
    #                 boxes=None,
    #                 masks=None,
    #             )
    #             pred, _ = sam.mask_decoder(
    #                             image_embeddings=img_emb,
    #                             image_pe=sam.prompt_encoder.get_dense_pe(), 
    #                             sparse_prompt_embeddings=sparse_emb,
    #                             dense_prompt_embeddings=dense_emb, 
    #                             multimask_output=True,
    #                             )
    #             loss = criterion1(pred,msks.float()) + criterion2(pred,torch.squeeze(msks.long(),1))
    #             eval_loss +=loss.item()
    #             dsc_batch = dice_coeff_multi_class(pred.argmax(dim=1).cpu(), torch.squeeze(msks.long(),1).cpu().long(),args.num_cls)
    #             dsc+=dsc_batch

    #         eval_loss /= (i+1)
    #         dsc /= (i+1)
            
    #         writer.add_scalar('eval/loss', eval_loss, epoch)
    #         writer.add_scalar('eval/dice', dsc, epoch)
            
    #         print('Eval Epoch num {} | val loss {} | dsc {} \n'.format(epoch,eval_loss,dsc))
    #         if dsc>val_largest_dsc:
    #             val_largest_dsc = dsc
    #             last_update_epoch = epoch
    #             print('largest DSC now: {}'.format(dsc))
    #             torch.save(sam.state_dict(),checkpoints_path + '/checkpoint_best.pth')
    #         elif (epoch-last_update_epoch)>20:
    #             # the network haven't been updated for 20 epochs
    #             print('Training finished###########')
    #             break
writer.close()

if update encoder: False
if image encoder adapter: False
if mask decoder adapter: True


  0%|          | 0/80 [00:00<?, ?it/s]


RuntimeError: Given groups=1, weight of size [32, 3, 3, 3], expected input[1, 1024, 1024, 3] to have 3 channels, but got 1024 channels instead